In [ ]:
!pip install gensim

In [2]:
!pip install tensorflow==2.18.0 keras==3.8.0 numpy==1.26.4 --no-cache-dir --upgrade --trusted-host pypi.org --trusted-host files.pythonhosted.org


In [3]:
!pip install Sastrawi

In [4]:
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from gensim.models import Word2Vec, FastText

import pickle
from keras.models import load_model
import joblib



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Mengambil dataset CSV TLKM dari GitHub (raw URL)
url = 'https://raw.githubusercontent.com/2209106126-azahrinathirah/Submission-Coding-Camp/refs/heads/main/Belajar%20Fundamental%20Deep%20Learning/Proyek%20Pertama/Dataset/livin_mandiri_reviews.csv'

# Membaca dataset
df = pd.read_csv(url)

# Menampilkan 5 dataset (5 baris pertama dan 5 baris terakhir)
df.head()


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,b40841be-1017-4394-94b5-bfd2b0849482,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kenapa apk livin sekang tidak bisa scan QRIS a...,2,0,3.0.0,2025-06-29 00:26:54,"Halo Sahabat @Zaydun Jay, mohon maaf atas keti...",2025-06-29 00:31:09,3.0.0
1,0dcfa37b-0e2f-4de8-9e48-4d0a9f82f4ac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,mantap,5,0,NaN,2025-06-29 00:24:58,"Halo Sahabat @jamuddin jems, terima kasih atas...",2025-06-29 00:29:28,NaN
2,096007e2-4fa9-4bf4-82dd-47450197026d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,gak bisa mendaftar gagal terus,1,0,3.0.0,2025-06-29 00:21:16,"Halo Sahabat @rendi cules, mohon maaf atas ket...",2025-06-29 00:24:37,3.0.0
3,fac388d5-b5db-43ea-b194-b87c65cc6c32,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat bermanfaat,5,0,3.0.0,2025-06-29 00:05:13,"Halo Sahabat @Ibnu Fadillah, terima kasih atas...",2025-06-29 00:09:38,3.0.0
4,02d58f18-9c11-439c-a41d-ae32f046aff2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sering mengalami tabungan 0 padahal habis dpt ...,2,0,3.0.0,2025-06-29 00:03:36,"Halo Sahabat @chintya maratus, mohon maaf atas...",2025-06-29 00:08:18,3.0.0


In [6]:
missing_content = df['content'].isnull().sum()
print(f"Jumlah missing value pada kolom 'content': {missing_content}")

Jumlah missing value pada kolom 'content': 0


In [7]:
jumlah_duplikat = df.duplicated().sum()
print(f"Jumlah duplikat: {jumlah_duplikat}")

Jumlah duplikat: 0


In [8]:
# Inisialisasi stemmer
stemmer = StemmerFactory().create_stemmer()

# Kamus slang lengkap
slang_dict = {
    "gk": "tidak", "ga": "tidak", "tdk": "tidak", "gak": "tidak", "ngga": "tidak",
    "bgt": "banget", "bngt": "banget",
    "dgn": "dengan", "sm": "sama", "ama": "sama",
    "udh": "sudah", "sdh": "sudah", "dh": "sudah",
    "blm": "belum", "blom": "belum",
    "aja": "saja", "aj": "saja", "ja": "saja",
    "yg": "yang", "ny": "nya", "nyg": "nya", "nya": "nya",
    "dr": "dari", "dar": "dari",
    "tp": "tapi", "tpi": "tapi",
    "mo": "mau", "mauu": "mau",
    "lg": "lagi", "lgi": "lagi",
    "kl": "kalau", "klo": "kalau", "kalo": "kalau",
    "trs": "terus", "trus": "terus",
    "ok": "oke", "sip": "oke",
    "bkn": "bukan", "bukn": "bukan",
    "jg": "juga", "jga": "juga",
    "cm": "cuma", "doang": "saja",
    "bbrp": "beberapa", "brp": "berapa",
    "masi": "masih", "msh": "masih",
    "bnr": "benar", "bener": "benar", "bner": "benar",
    "bru": "baru", "br": "baru",
    "apk": "aplikasi",
    "knp": "kenapa",
    "tf": "transfer", "trf": "transfer",
    "mending": "lebih baik",
    "musti": "harus",
    "sprt": "seperti"
}

# Gabungkan stopwords dari NLTK bahasa Indonesia dan Inggris, plus tambahan
stop_words = set(stopwords.words('indonesian')) | set(stopwords.words('english'))
stop_words.update({'dan', 'yang', 'di', 'ke', 'nya', 'ini', 'itu' 'lah', 'ya', 'deh', 'ko', 'emang'})

# Kata penting yang tidak boleh dihapus
important_short_words = {'tidak', 'oke', 'bagus', 'top', 'cukup', 'baik', 'sangat', 'bisa', 'baru', 'lebih'}


def reduce_repeated_characters(text):
    # Ganti 3 atau lebih karakter berulang jadi hanya 1
    return re.sub(r'(.)\1{2,}', r'\1', text)

# Fungsi case folding
def case_folding(text):
    return text.lower()

# Fungsi normalisasi slang
def normalize_slang(text):
    words = text.split()
    return ' '.join([slang_dict.get(word, word) for word in words])

# Fungsi preprocessing lengkap
def clean_text(text):
    text = str(text)

    # 1. Case folding
    text = text.lower()

    # 2. Hapus angka
    text = re.sub(r'\d+', '', text)

    # 3. Hapus tanda baca, tapi jaga markdown (*, _, ~)
    text = re.sub(r'[^\w\s\*\_\~]', '', text)

    # 4. Hapus karakter non-ASCII, tapi jaga markdown
    text = re.sub(r'[^\x00-\x7F\*\_\~]+', ' ', text)

    # 5. Hapus spasi ganda
    text = re.sub(r'\s+', ' ', text).strip()

    # 6. Reduksi karakter berulang
    text = reduce_repeated_characters(text)

    # 7. Split kata
    words = text.split()

    # 8. Normalisasi slang, tapi jangan ubah kata penting
    words = [
        slang_dict.get(word, word) if word not in important_short_words else word
        for word in words
    ]

    # 9. Stopword removal tapi jaga kata penting
    words = [word for word in words if word in important_short_words or word not in stop_words]

    # words = [
    # slang_dict.get(word, word)
    # for word in words
    # ]

    # 10. Stemming, tapi hindari kata dengan markdown
    stemmed_words = []
    for word in words:
        if any(char in word for char in ['*', '_', '~']):
            stemmed_words.append(word)
        else:
            stemmed_words.append(stemmer.stem(word))

    return ' '.join(stemmed_words)

In [9]:
# Terapkan ke data
df['cleaned_ulasan'] = df['content'].astype(str).apply(clean_text)

# Rename kolom dan tampilkan
df_clean = df[['content', 'cleaned_ulasan']].rename(columns={
    'content': 'ulasan asli',
    'cleaned_ulasan': 'ulasan bersih'
})

df_clean.head(10)

,ulasan asli,ulasan bersih
0,kenapa apk livin sekang tidak bisa scan QRIS a...,aplikasi livin sekang tidak bisa scan qris ane...
1,mantap,mantap
2,gak bisa mendaftar gagal terus,tidak bisa daftar gagal
3,sangat bermanfaat,sangat manfaat
4,sering mengalami tabungan 0 padahal habis dpt ...,alami tabung habis dpt transfer
5,otp nya lama,otp
6,sangat cepat dan mudah dipakai,sangat cepat mudah pakai
7,"kenapa ini tidak bisa buka aplikasi livin ,,ah...",tidak bisa buka aplikasi livin ah apaa via email
8,gmna si ngirim kode OTP ko lama bener sampe wa...,gmna si ngirim kode otp sampe abis baru kirim ...
9,"tidak bisa login,error terus,kacau nih aplikasi",tidak bisa loginerror teruskacau nih aplikasi


In [10]:
# --- Stopwords ---
stop_words = ['dan', 'di', 'yang', 'ke', 'dari', 'untuk', 'dengan', 'adalah', 'ini', 'itu']

# --- 1. Labeling Sentimen Manual dengan Deteksi Konteks Negatif ---
def label_sentiment_manual(text):
    text = str(text).lower()
    positif_words = ['bagus', 'suka', 'mantap', 'lancar', 'cepat', 'baik', 'top', 'puas', 'cocok', 'senang', 'terbaik', 'mantul', 'recommended']
    negatif_words = ['jelek', 'gagal', 'error', 'tidak bisa', 'buruk', 'lemot', 'tidak sesuai', 'kecewa', 'parah', 'lambat', 'sampah', 'penipuan']
    text_clean = text.translate(str.maketrans('', '', string.punctuation))

    # Cek frasa panjang terlebih dahulu
    neg_count = sum(phrase in text_clean for phrase in negatif_words)
    pos_count = sum(word in text_clean.split() for word in positif_words)

    if pos_count > neg_count:
        return 'positif'
    elif neg_count > pos_count:
        return 'negatif'
    else:
        return 'netral'

# --- 2. Labelkan Data ---
df_clean['sentimen'] = df_clean['ulasan bersih'].apply(label_sentiment_manual)


In [11]:
print(df_clean['sentimen'].value_counts())


sentimen
netral     5625
positif    2255
negatif    2120
Name: count, dtype: int64


In [27]:
df_clean[['ulasan bersih', 'sentimen']].to_csv("ulasan_dan_sentimen.csv", index=False)

## Skema 1: LSTM + Word2Vec (80/20)

In [12]:
# Buat kolom 'tokens'
df_clean['tokens'] = df_clean['ulasan bersih'].apply(
    lambda x: str(x).lower().translate(str.maketrans('', '', string.punctuation)).split()
)

# --- 1. Latih Word2Vec ---
w2v_model = Word2Vec(sentences=df_clean['tokens'], vector_size=100, window=5, min_count=1, workers=4)
w2v_model.train(df_clean['tokens'], total_examples=len(df_clean['tokens']), epochs=10)

# --- 2. Tokenizer dan Padding ---
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_clean['ulasan bersih'])
sequences = tokenizer.texts_to_sequences(df_clean['ulasan bersih'])
word_index = tokenizer.word_index
X = pad_sequences(sequences, maxlen=100)

# --- 3. Label Encoding ---
le = LabelEncoder()
y = le.fit_transform(df_clean['sentimen'])
y_cat = to_categorical(y)

# --- 7. Split 80:20 ---
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y, random_state=42)

# --- 8. Embedding Matrix dari Word2Vec ---
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# --- 9. Model LSTM ---
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=100,
                    weights=[embedding_matrix],
                    input_length=100,
                    trainable=False))  # Word2Vec tidak dilatih ulang
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- 10. EarlyStopping ---
es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# --- 11. Training ---
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=64,
                    callbacks=[es],
                    verbose=1)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 172ms/step - accuracy: 0.5793 - loss: 0.8865 - val_accuracy: 0.6950 - val_loss: 0.6775
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 171ms/step - accuracy: 0.7153 - loss: 0.6477 - val_accuracy: 0.7638 - val_loss: 0.5541
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 181ms/step - accuracy: 0.7583 - loss: 0.5497 - val_accuracy: 0.7656 - val_loss: 0.5103
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 183ms/step - accuracy: 0.8052 - loss: 0.4771 - val_accuracy: 0.8206 - val_loss: 0.4483
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 177ms/step - accuracy: 0.8383 - loss: 0.4066 - val_accuracy: 0.8669 - val_loss: 0.3934
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 170ms/step - accuracy: 0.8647 - loss: 0.3612 - val_accuracy: 0.8737 - val_loss: 0.3505
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 22s 183ms/step - accuracy: 0.8771 - loss: 0.3255 - val_accuracy: 0.8781 - val_loss: 0.3367
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 187ms/step - accuracy: 0.8960 - loss: 0.2860 - val

In [13]:
# --- Evaluasi untuk Training Set ---
y_train_pred_prob = model.predict(X_train)
y_train_pred = y_train_pred_prob.argmax(axis=1)
y_train_labels = y_train.argmax(axis=1)

train_acc = accuracy_score(y_train_labels, y_train_pred) * 100
print("Accuracy Training: {:.2f}%".format(train_acc))
print(classification_report(y_train_labels, y_train_pred, target_names=le.classes_))

# --- Evaluasi untuk Testing Set ---
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)
y_test_labels = y_test.argmax(axis=1)

test_acc = accuracy_score(y_test_labels, y_pred) * 100
print("Accuracy Testing : {:.2f}%".format(test_acc))
print(classification_report(y_test_labels, y_pred, target_names=le.classes_))


250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step
Accuracy Training: 92.90%
              precision    recall  f1-score   support

     negatif       0.98      0.80      0.88      1696
      netral       0.91      0.97      0.94      4500
     positif       0.94      0.94      0.94      1804

    accuracy                           0.93      8000
   macro avg       0.95      0.90      0.92      8000
weighted avg       0.93      0.93      0.93      8000

63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step
Accuracy Testing : 93.00%
              precision    recall  f1-score   support

     negatif       0.99      0.83      0.90       424
      netral       0.91      0.97      0.94      1125
     positif       0.93      0.92      0.92       451

    accuracy                           0.93      2000
   macro avg       0.94      0.91      0.92      2000
weighted avg       0.93      0.93      0.93      2000



## Skema 2: LSTM + Embedding layer (70/30)

In [17]:
# --- 1. Tokenisasi dan Padding ---
MAX_FEATURES = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(df_clean['ulasan bersih'])

X_seq = tokenizer.texts_to_sequences(df_clean['ulasan bersih'])
X_pad = pad_sequences(X_seq, maxlen=MAX_LEN)

# --- 2. Label Encoding dan One-Hot ---
le = LabelEncoder()
y = le.fit_transform(df_clean['sentimen'])
y_cat = to_categorical(y)

# --- 3. Split Data 70/30 ---
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_cat, test_size=0.3, stratify=y, random_state=42)

# --- 4. Bangun Model LSTM ---
model = Sequential()
model.add(Embedding(MAX_FEATURES, 128, input_length=MAX_LEN))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # 3 kelas sentimen

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- 5. Training dengan EarlyStopping ---
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


88/88 ━━━━━━━━━━━━━━━━━━━━ 27s 195ms/step - accuracy: 0.5660 - loss: 0.9249 - val_accuracy: 0.8214 - val_loss: 0.4633
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.8470 - loss: 0.3897 - val_accuracy: 0.9071 - val_loss: 0.2646
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 21s 171ms/step - accuracy: 0.9409 - loss: 0.1887 - val_accuracy: 0.9436 - val_loss: 0.1851
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 164ms/step - accuracy: 0.9669 - loss: 0.1044 - val_accuracy: 0.9500 - val_loss: 0.1624
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 15s 170ms/step - accuracy: 0.9768 - loss: 0.0730 - val_accuracy: 0.9493 - val_loss: 0.1647
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.9846 - loss: 0.0523 - val_accuracy: 0.9464 - val_loss: 0.1707
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 4.


In [18]:
# --- Evaluasi untuk Training Set ---
y_train_pred_prob = model.predict(X_train)
y_train_pred = y_train_pred_prob.argmax(axis=1)
y_train_labels = y_train.argmax(axis=1)

train_acc = accuracy_score(y_train_labels, y_train_pred) * 100
print("Accuracy Training: {:.2f}%".format(train_acc))
print(classification_report(y_train_labels, y_train_pred, target_names=le.classes_))

# --- Evaluasi untuk Testing Set ---
y_test_pred_prob = model.predict(X_test)
y_test_pred = y_test_pred_prob.argmax(axis=1)
y_test_labels = y_test.argmax(axis=1)

test_acc = accuracy_score(y_test_labels, y_test_pred) * 100
print("Accuracy Testing : {:.2f}%".format(test_acc))
print(classification_report(y_test_labels, y_test_pred, target_names=le.classes_))


219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step
Accuracy Training: 97.69%
              precision    recall  f1-score   support

     negatif       0.96      0.97      0.96      1484
      netral       0.98      0.97      0.98      3937
     positif       0.98      0.99      0.98      1579

    accuracy                           0.98      7000
   macro avg       0.97      0.98      0.98      7000
weighted avg       0.98      0.98      0.98      7000

94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step
Accuracy Testing : 95.30%
              precision    recall  f1-score   support

     negatif       0.91      0.94      0.93       636
      netral       0.97      0.95      0.96      1688
     positif       0.95      0.98      0.96       676

    accuracy                           0.95      3000
   macro avg       0.95      0.96      0.95      3000
weighted avg       0.95      0.95      0.95      3000



## Skema 3: CNN + FastText (70/30)

In [19]:
# --- 1. Preprocessing Token ---
df_clean['tokens'] = df_clean['ulasan bersih'].apply(
    lambda x: str(x).lower().translate(str.maketrans('', '', string.punctuation)).split()
)

# --- 2. Train FastText Model ---
ft_model = FastText(sentences=df_clean['tokens'], vector_size=100, window=5, min_count=1, workers=4, sg=1)
ft_model.train(df_clean['tokens'], total_examples=len(df_clean['tokens']), epochs=10)

# --- 3. Tokenizer dan Padding ---
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_clean['ulasan bersih'])
sequences = tokenizer.texts_to_sequences(df_clean['ulasan bersih'])
word_index = tokenizer.word_index
X = pad_sequences(sequences, maxlen=100)

# --- 4. Label Encoding ---
le = LabelEncoder()
y = le.fit_transform(df_clean['sentimen'])
y_cat = to_categorical(y)

# --- 5. Split Data 70/30 ---
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.3, stratify=y, random_state=42)

# --- 6. Embedding Matrix dari FastText ---
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    if word in ft_model.wv:
        embedding_matrix[i] = ft_model.wv[word]

# --- 7. Model CNN ---
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=100,
                    weights=[embedding_matrix],
                    input_length=100,
                    trainable=False))  # FastText embeddings (non-trainable)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- 8. Training ---
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=64,
                    callbacks=[early_stop],
                    verbose=1)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.5985 - loss: 0.8971 - val_accuracy: 0.7829 - val_loss: 0.5738
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.7647 - loss: 0.5992 - val_accuracy: 0.8471 - val_loss: 0.4220
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.8322 - loss: 0.4459 - val_accuracy: 0.8921 - val_loss: 0.3216
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.8605 - loss: 0.3701 - val_accuracy: 0.8871 - val_loss: 0.3112
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.8762 - loss: 0.3223 - val_accuracy: 0.9193 - val_loss: 0.2496
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.8965 - loss: 0.2759 - val_accuracy: 0.9257 - val_loss: 0.2262
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.9104 - loss: 0.2374 - val_accuracy: 0.9307 - val_loss: 0.2249
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9178 - loss: 0.2257 - val_accuracy: 0.9300 - val_loss: 0.

In [20]:
# --- Evaluasi untuk Training Set ---
y_train_pred_prob = model.predict(X_train)
y_train_pred = y_train_pred_prob.argmax(axis=1)
y_train_labels = y_train.argmax(axis=1)

train_acc = accuracy_score(y_train_labels, y_train_pred) * 100
print("Accuracy Training: {:.2f}%".format(train_acc))
print(classification_report(y_train_labels, y_train_pred, target_names=le.classes_))

# --- Evaluasi untuk Testing Set ---
y_test_pred_prob = model.predict(X_test)
y_test_pred = y_test_pred_prob.argmax(axis=1)
y_test_labels = y_test.argmax(axis=1)

test_acc = accuracy_score(y_test_labels, y_test_pred) * 100
print("Accuracy Testing : {:.2f}%".format(test_acc))
print(classification_report(y_test_labels, y_test_pred, target_names=le.classes_))



219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Accuracy Training: 95.99%
              precision    recall  f1-score   support

     negatif       0.98      0.91      0.94      1484
      netral       0.95      0.98      0.96      3937
     positif       0.97      0.96      0.96      1579

    accuracy                           0.96      7000
   macro avg       0.97      0.95      0.96      7000
weighted avg       0.96      0.96      0.96      7000

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Accuracy Testing : 94.93%
              precision    recall  f1-score   support

     negatif       0.97      0.91      0.94       636
      netral       0.94      0.97      0.96      1688
     positif       0.95      0.93      0.94       676

    accuracy                           0.95      3000
   macro avg       0.95      0.94      0.95      3000
weighted avg       0.95      0.95      0.95      3000



In [21]:
# Simpan LabelEncoder ke file
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("✅ LabelEncoder berhasil disimpan sebagai 'label_encoder.pkl'")

✅ LabelEncoder berhasil disimpan sebagai 'label_encoder.pkl'


In [22]:
with open("tokenizer_lstm_w2v.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Tokenizer skema LSTM + Word2Vec berhasil disimpan.")

with open("tokenizer_lstm_embed.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Tokenizer skema LSTM + Embedding berhasil disimpan.")

with open("tokenizer_cnn_ft.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Tokenizer untuk CNN + FastText berhasil disimpan.")


✅ Tokenizer skema LSTM + Word2Vec berhasil disimpan.
✅ Tokenizer skema LSTM + Embedding berhasil disimpan.
✅ Tokenizer untuk CNN + FastText berhasil disimpan.


In [23]:
model.save("model_lstm_w2v.h5")
print("✅ Model LSTM + Word2Vec berhasil disimpan.")

model.save("model_lstm_embed.h5")
print("✅ Model LSTM + Embedding berhasil disimpan.")

model.save("model_cnn_ft.h5")
print("✅ Model CNN + FastText berhasil disimpan.")

✅ Model LSTM + Word2Vec berhasil disimpan.
✅ Model LSTM + Embedding berhasil disimpan.
✅ Model CNN + FastText berhasil disimpan.


In [25]:
# Kalimat uji
test_texts = {
    "Negatif": "Setelah update, aplikasi jadi rusak total dan tidak bisa dipakai",
    "Netral" : "Saya melakukan transfer dana menggunakan aplikasi Livin",
    "Positif": "Aplikasinya bagus banget dan sangat memudahkan transaksi"
}

# Skema 1: LSTM + Word2Vec
with open("tokenizer_lstm_w2v.pkl", "rb") as f:
    tokenizer_w2v = pickle.load(f)
model_lstm_w2v = load_model("model_lstm_w2v.h5")
def predict_lstm_w2v(text):
    seq = tokenizer_w2v.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=100)
    pred = model_lstm_w2v.predict(pad)
    class_idx = np.argmax(pred, axis=1)[0]
    # Transform indeks ke label asli
    label = le.inverse_transform([class_idx])[0]
    return label

# Skema 2: LSTM + Embedding
with open("tokenizer_lstm_embed.pkl", "rb") as f:
    tokenizer_embed = pickle.load(f)
model_lstm_embed = load_model("model_lstm_embed.h5")
def predict_lstm_embed(text):
    seq = tokenizer_embed.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=100)
    pred = model_lstm_embed.predict(pad)
    return le.inverse_transform([np.argmax(pred)])[0]

# Skema 3: CNN + FastText
with open("tokenizer_cnn_ft.pkl", "rb") as f:
    tokenizer_cnn = pickle.load(f)
model_cnn_ft = load_model("model_cnn_ft.h5")
def predict_cnn_ft(text):
    seq = tokenizer_cnn.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=100)
    pred = model_cnn_ft.predict(pad)
    return le.inverse_transform([np.argmax(pred)])[0]


print("=== Hasil Prediksi Setiap Kalimat ===\n")
for label, kalimat in test_texts.items():
    print(f"Kalimat ({label}): {kalimat}")
    # print(f"  TF-IDF + Dense        : {predict_tfidf(kalimat)}")
    print(f"  LSTM + Word2Vec       : {predict_lstm_w2v(kalimat)}")
    print(f"  LSTM + Embedding      : {predict_lstm_embed(kalimat)}")
    print(f"  CNN + FastText        : {predict_cnn_ft(kalimat)}\n")



=== Hasil Prediksi Setiap Kalimat ===

Kalimat (Negatif): Setelah update, aplikasi jadi rusak total dan tidak bisa dipakai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
  LSTM + Word2Vec       : negatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
  LSTM + Embedding      : negatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
  CNN + FastText        : negatif

Kalimat (Netral): Saya melakukan transfer dana menggunakan aplikasi Livin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
  LSTM + Word2Vec       : netral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
  LSTM + Embedding      : netral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
  CNN + FastText        : netral

Kalimat (Positif): Aplikasinya bagus banget dan sangat memudahkan transaksi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
  LSTM + Word2Vec       : positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
  LSTM + Embedding      : positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
  CNN + FastText        : positif



In [ ]:
!pip freeze > requirements.txt